In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Trafic SNCF") \
    .getOrCreate()


In [2]:
chemin_delta = "/home/jovyan/work/data/delta_sncf"

df_trafic = spark.read.format("delta").load(chemin_delta)

df_trafic.show(5)
df_trafic.printSchema()


+--------------------+----------+--------------------+----------+----------+
|             trip_id|start_date|             stop_id|   arrival| departure|
+--------------------+----------+--------------------+----------+----------+
|OCESN3604F4279771...|  20250522|StopPoint:OCEINTE...|      null|1747878900|
|OCESN3604F4279771...|  20250522|StopPoint:OCEINTE...|1747882500|1747882680|
|OCESN3604F4279771...|  20250522|StopPoint:OCEINTE...|1747884540|1747884600|
|OCESN3604F4279771...|  20250522|StopPoint:OCEINTE...|1747887060|1747887180|
|OCESN3604F4279771...|  20250522|StopPoint:OCEINTE...|1747888020|1747888140|
+--------------------+----------+--------------------+----------+----------+
only showing top 5 rows

root
 |-- trip_id: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- arrival: long (nullable = true)
 |-- departure: long (nullable = true)



In [4]:
# Charger les noms des arrêts depuis le fichier CSV
chemin_stops = "/home/jovyan/work/data/Exemple_stops_csv.csv"
df_stops = spark.read.option("header", True).csv(chemin_stops)

# Jointure avec les noms des arrêts
df_trafic = df_trafic.join(df_stops, on="stop_id", how="left")


In [5]:
from pyspark.sql.functions import col, from_unixtime

df_trafic = df_trafic.withColumn("heure_arrivee", from_unixtime(col("arrival"))) \
                     .withColumn("heure_depart", from_unixtime(col("departure")))

df_trafic.select("stop_name", "heure_arrivee", "heure_depart").show(5, truncate=False)


+---------+-------------------+-------------------+
|stop_name|heure_arrivee      |heure_depart       |
+---------+-------------------+-------------------+
|null     |null               |2025-05-22 01:55:00|
|null     |2025-05-22 02:55:00|2025-05-22 02:58:00|
|null     |2025-05-22 03:29:00|2025-05-22 03:30:00|
|null     |2025-05-22 04:11:00|2025-05-22 04:13:00|
|null     |2025-05-22 04:27:00|2025-05-22 04:29:00|
+---------+-------------------+-------------------+
only showing top 5 rows



In [7]:
chemin_stops = "/home/jovyan/work/data/Exemple_stops_csv.csv"
df_stops = spark.read.option("header", True).csv(chemin_stops)

df_stops.show(5, truncate=False)
df_stops.printSchema()


+-------------------------+------------------+
|stop_id                  |stop_name         |
+-------------------------+------------------+
|StopPoint:OCEICE-80110684|Paris Gare de Lyon|
|StopPoint:OCEICE-80140087|Melun             |
|StopPoint:OCEICE-80140210|Fontainebleau     |
|StopPoint:OCEICE-80142281|Montereau         |
|StopPoint:OCEICE-80142778|Sens              |
+-------------------------+------------------+
only showing top 5 rows

root
 |-- stop_id: string (nullable = true)
 |-- stop_name: string (nullable = true)



In [9]:
df_joint.select(
    df_joint["stop_id"],
    df_stops["stop_name"],  # On précise qu'on prend celui du CSV
    df_joint["arrival"],
    df_joint["departure"]
).show(10, truncate=False)


+--------------------------------+---------+----------+----------+
|stop_id                         |stop_name|arrival   |departure |
+--------------------------------+---------+----------+----------+
|StopPoint:OCEINTERCITES-87594002|null     |null      |1747878900|
|StopPoint:OCEINTERCITES-87592006|null     |1747882500|1747882680|
|StopPoint:OCEINTERCITES-87592378|null     |1747884540|1747884600|
|StopPoint:OCEINTERCITES-87597005|null     |1747887060|1747887180|
|StopPoint:OCEINTERCITES-87597062|null     |1747888020|1747888140|
|StopPoint:OCEINTERCITES-87576009|null     |1747889340|1747889460|
|StopPoint:OCEINTERCITES-87547000|null     |1747895460|null      |
|StopPoint:OCETrain TER-87212027 |null     |null      |1747881420|
|StopPoint:OCETrain TER-87212225 |null     |1747882740|1747882800|
|StopPoint:OCETrain TER-87175042 |null     |1747890000|1747890120|
+--------------------------------+---------+----------+----------+
only showing top 10 rows



In [10]:
df_stops = df_stops.withColumnRenamed("stop_name", "nom_arret")
df_joint = df_trafic.join(df_stops, on="stop_id", how="left")

df_joint.select("stop_id", "nom_arret", "arrival", "departure").show(10, truncate=False)


+--------------------------------+---------+----------+----------+
|stop_id                         |nom_arret|arrival   |departure |
+--------------------------------+---------+----------+----------+
|StopPoint:OCEINTERCITES-87594002|null     |null      |1747878900|
|StopPoint:OCEINTERCITES-87592006|null     |1747882500|1747882680|
|StopPoint:OCEINTERCITES-87592378|null     |1747884540|1747884600|
|StopPoint:OCEINTERCITES-87597005|null     |1747887060|1747887180|
|StopPoint:OCEINTERCITES-87597062|null     |1747888020|1747888140|
|StopPoint:OCEINTERCITES-87576009|null     |1747889340|1747889460|
|StopPoint:OCEINTERCITES-87547000|null     |1747895460|null      |
|StopPoint:OCETrain TER-87212027 |null     |null      |1747881420|
|StopPoint:OCETrain TER-87212225 |null     |1747882740|1747882800|
|StopPoint:OCETrain TER-87175042 |null     |1747890000|1747890120|
+--------------------------------+---------+----------+----------+
only showing top 10 rows



In [12]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/home/jovyan/.kaggle"


# Charger les données des gares

In [14]:
import pandas as pd
df_gares = pd.read_csv("/home/jovyan/work/data/gares_kaggle.csv")
df_gares["Code UIC"] = df_gares["Code UIC"].astype("Int64")
df_gares["Code UIC"] = df_gares["Code UIC"].astype(str)
df_gares["Code UIC"] = df_gares["Code UIC"].str.zfill(8)
df_gares["stop_id"] = "StopPoint:OCE" + df_gares["Code UIC"]


In [ ]:
# df_gares_spark = spark.createDataFrame(df_gares[["stop_id", "Libellé Gare"]].rename(columns={"Libellé Gare": "stop_name"}))


In [15]:
df_gares_spark = spark.createDataFrame(df_gares[["stop_id", "Libellé Gare"]].rename(columns={"Libellé Gare": "stop_name"}))


In [16]:
# Joindre avec df_trafic
df_trafic = df_trafic.drop("stop_name") if "stop_name" in df_trafic.columns else df_trafic
df_trafic = df_trafic.join(df_gares_spark, on="stop_id", how="left")


In [18]:
#Vérifier le résultat
df_trafic.select("stop_id", "stop_name", "arrival", "departure").show(30, truncate=False)


+--------------------------------+---------+----------+----------+
|stop_id                         |stop_name|arrival   |departure |
+--------------------------------+---------+----------+----------+
|StopPoint:OCEINTERCITES-87597005|null     |1747887060|1747887180|
|StopPoint:OCETrain TER-87673400 |null     |1747883640|1747883700|
|StopPoint:OCETrain TER-87212027 |null     |null      |1747881420|
|StopPoint:OCETrain TER-87113001 |null     |1747900380|null      |
|StopPoint:OCETrain TER-87175042 |null     |1747890000|1747890120|
|StopPoint:OCETrain TER-87673277 |null     |1747885380|1747885440|
|StopPoint:OCETrain TER-87581009 |null     |1747892280|null      |
|StopPoint:OCETrain TER-87171553 |null     |1747895220|1747895400|
|StopPoint:OCETrain TER-87673103 |null     |1747888080|1747888140|
|StopPoint:OCETrain TER-87673459 |null     |1747883280|1747883340|
|StopPoint:OCETrain TER-87582536 |null     |1747889220|1747889280|
|StopPoint:OCETrain TER-87212225 |null     |1747882740|1747882

In [20]:
# 1. Import des bibliothèques
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, regexp_extract
import pandas as pd

# 2. Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("Enrichissement des noms de gares") \
    .getOrCreate()

# 3. Chargement du fichier Delta contenant les données de trafic
chemin_delta = "/home/jovyan/work/data/delta_sncf"
df_trafic = spark.read.format("delta").load(chemin_delta)

# 4. Ajout des colonnes lisibles pour les horaires
df_trafic = df_trafic \
    .withColumn("heure_arrivee", from_unixtime(col("arrival"))) \
    .withColumn("heure_depart", from_unixtime(col("departure")))

# 5. Extraction du code UIC depuis stop_id
df_trafic = df_trafic.withColumn("uic_code", regexp_extract("stop_id", r"(\d{8})", 1))

# 6. Chargement des données Kaggle sur les gares
df_gares = pd.read_csv("/home/jovyan/work/data/gares_kaggle.csv")

# 7. Nettoyage des codes UIC (complétés à 8 chiffres)
df_gares["Code UIC"] = df_gares["Code UIC"].astype("Int64").astype(str).str.zfill(8)

# 8. Sélection et renommage des colonnes utiles
df_gares_spark = spark.createDataFrame(
    df_gares[["Code UIC", "Libellé Gare"]].rename(
        columns={"Code UIC": "uic_code", "Libellé Gare": "stop_name"}
    )
)

# 9. Jointure entre les données trafic et les noms de gares
df_trafic = df_trafic.join(df_gares_spark, on="uic_code", how="left")

# 10. Aperçu du résultat enrichi
df_trafic.select("stop_id", "stop_name", "heure_arrivee", "heure_depart").show(10, truncate=False)

# 11. Export du résultat pour Power BI (si besoin)
df_trafic.write.mode("overwrite").option("header", True).csv("/home/jovyan/work/export_powerbi/trafic_enrichi")


+-------------------------------+-----------------+-------------------+-------------------+
|stop_id                        |stop_name        |heure_arrivee      |heure_depart       |
+-------------------------------+-----------------+-------------------+-------------------+
|StopPoint:OCETrain TER-87174276|Vitry-le-François|2025-05-22 05:47:00|2025-05-22 05:50:00|
|StopPoint:OCETrain TER-87174276|Vitry-le-François|2025-05-22 05:47:00|2025-05-22 05:50:00|
|StopPoint:OCETrain TER-87174276|Vitry-le-François|2025-05-22 05:47:00|2025-05-22 05:50:00|
|StopPoint:OCETrain TER-87175042|Bar-le-Duc       |2025-05-22 05:00:00|2025-05-22 05:02:00|
|StopPoint:OCETrain TER-87212027|Strasbourg-Ville |null               |2025-05-22 02:37:00|
|StopPoint:OCETrain TER-87212027|Strasbourg-Ville |null               |2025-05-22 02:37:00|
|StopPoint:OCETrain TER-87212027|Strasbourg-Ville |null               |2025-05-22 02:37:00|
|StopPoint:OCETrain TER-87212027|Strasbourg-Ville |null               |2025-05-2